In [2]:
import pandas as pd
from pathlib import Path
from geopy.distance import geodesic

# Load your Uber rides data into a DataFrame
df = pd.read_csv("Resources/uber.csv")

# Filter and clean the DataFrame
df_filtered = df[(df['fare_amount'] >= 0) & 
                 (df['pickup_longitude'] != 0) & 
                 (df['pickup_latitude'] != 0) & 
                 (df['dropoff_longitude'] != 0) & 
                 (df['dropoff_latitude'] != 0) & 
                 (df['passenger_count'] != 0)]

# Sort the DataFrame based on passenger_count
df_sorted = df_filtered.sort_values(by='passenger_count')

# Calculate road distance using geopy
def calculate_distance(row):
    pickup_latitude, pickup_longitude = row['pickup_latitude'], row['pickup_longitude']
    dropoff_latitude, dropoff_longitude = row['dropoff_latitude'], row['dropoff_longitude']
    
    # Check for valid latitude values
    if not (-90 <= pickup_latitude <= 90) or not (-90 <= dropoff_latitude <= 90):
        return None  # Return None for rows with invalid latitude
    
    # Swap latitude and longitude to ensure correct order
    pickup_coords = (pickup_latitude, pickup_longitude)
    dropoff_coords = (dropoff_latitude, dropoff_longitude)
    
    # Calculate distance if latitude values are valid
    distance = geodesic(pickup_coords, dropoff_coords).miles
    
    return distance

df_sorted['road_distance'] = df_sorted.apply(calculate_distance, axis=1)

# Drop the original 'key', 'pickup_datetime', and 'Unnamed: 0' columns
df_sorted = df_sorted.drop(columns=['key', 'pickup_datetime', 'Unnamed: 0'])

# Save the modified DataFrame to a new Excel file
df_sorted.to_excel('uber_rides_distance_calculated_geopy.xlsx', index=False)